In [1]:
pwd

'/Users/kelly/metis_v3/Project_5'

In [14]:
import pprint

In [155]:
import nltk
from nltk.corpus import stopwords
import gensim

In [2]:
from pymongo import MongoClient
client = MongoClient()
db = client.goodreads

In [3]:
db.books.count()

10000

In [4]:
db.ratings.count()

5976479

In [25]:
db.booktags.find_one()

{'_id': ObjectId('5b8885c921c3f05eea553c1d'),
 'goodreads_book_id': 1,
 'tag_id': 30574,
 'count': 167697}

In [6]:
db.tags.count()

34252

In [7]:
db.books.find_one()

{'_id': ObjectId('5b88855b21c3f05eeaf945eb'),
 'book_id': 1,
 'goodreads_book_id': 2767052,
 'best_book_id': 2767052,
 'work_id': 2792775,
 'books_count': 272,
 'isbn': 439023483,
 'isbn13': 9780439023480.0,
 'authors': 'Suzanne Collins',
 'original_publication_year': 2008.0,
 'original_title': 'The Hunger Games',
 'title': 'The Hunger Games (The Hunger Games, #1)',
 'language_code': 'eng',
 'average_rating': 4.34,
 'ratings_count': 4780653,
 'work_ratings_count': 4942365,
 'work_text_reviews_count': 155254,
 'ratings_1': 66715,
 'ratings_2': 127936,
 'ratings_3': 560092,
 'ratings_4': 1481305,
 'ratings_5': 2706317,
 'image_url': 'https://images.gr-assets.com/books/1447303603m/2767052.jpg',
 'small_image_url': 'https://images.gr-assets.com/books/1447303603s/2767052.jpg'}

In [45]:
for book in db.booktags.aggregate([{'$sample':{'size':5},
                                    {'$group':{'goodreads_book_id':'id'}      
                                           
                                    }                                   
                                   }]):

    print (book)

SyntaxError: invalid syntax (<ipython-input-45-863cec9a7d59>, line 5)

In [47]:
for book in db.booktags.aggregate([{'$sample':{'size':5}},
                    {'$group': {'goodreads_book_id' : 'id' } }, 
                    {'$project':{'_id':0,'Book ID':'$book_id','Title':'$original_title'}}
                   ]):
        pprint.pprint(book)

OperationFailure: The field 'goodreads_book_id' must be an accumulator object

In [ ]:
def find_booktags(ID):
    booktag_list = []          
    for book in db.booktags.find({'goodreads_book_id':ID}):
        tagid = (book['tag_id'])
        for tag in db.tags.find({'tag_id':tagid}):
            booktag_list.append(tag['tag_name'].split('-'))
    return(booktag_list)

In [80]:
for book in db.books.aggregate([{'$sample':{'size':5}},{
    '$lookup': {
        'from': 'booktags',
        'localField': 'goodreads_book_id',
        'foreignField': 'goodreads_book_id',
        'as': 'tag_id'
    }
}, {
    
    '$unwind': '$booktags'  
}, {
    '$project': {
        'goodreads_book_id': '$books.goodreads_book_id',
        'tag_num': '$booktags.tag_id'
    }
}
]):
        pprint.pprint(book)

In [105]:
for book in db.booktags.aggregate([{'$sample':{'size':2}},
    {
        '$group': {
        'goodreads_book_id': '$goodreads_book_id',
     },
    }, {
    '$project': {
        'goodreads_book_id': '$booktags.goodreads_book_id',
        'tag_id': '$booktags.tag_id'
    }
        }
]):
    pprint.pprint(book)

OperationFailure: The field 'goodreads_book_id' must be an accumulator object

In [99]:
for book in db.books.aggregate([{'$sample':{'size':5}},
    {
        '$lookup': {
        'from': 'booktags',
        'localField': 'bookreads_book_id',
        'foreignField': 'tag_id',
        'as': 'monkey',
    }, 
    }, {
    '$project': {
        'goodreads_book_id': '$booktags.goodreads_book_id',
        'tag_num': '$booktags.tag_id'
    }    
}
]):
                                pprint.pprint(book)

{'_id': ObjectId('5b88855b21c3f05eeaf94c53')}
{'_id': ObjectId('5b88855b21c3f05eeaf950af')}
{'_id': ObjectId('5b88855c21c3f05eeaf96c6a')}
{'_id': ObjectId('5b88855b21c3f05eeaf94b82')}
{'_id': ObjectId('5b88855b21c3f05eeaf9504b')}


In [ ]:
db.votes.aggregate([{
    $lookup: {
        from: "users",
        localField: "createdBy",
        foreignField: "_id",
        as: "user"
    }
}, {
    $unwind: "$user"
}, {
    $unwind: "$user.emails"
}, {
    $sort: { "user.emails.verified": -1 }
}, {
    $group: {
        _id: "$_id",
        createdBy: { $first: "$createdBy" },
        fellowId: { $first: "$fellowId" },
        companyId: { $first: "$companyId" },
        teamId: { $first: "$teamId" },
        user: { $first: "$user" }
    }
}, {

In [72]:
def find_booktags(ID):
    booktag_list = []          
    for book in db.booktags.find({'goodreads_book_id':ID}):
        tagid = (book['tag_id'])
        for tag in db.tags.find({'tag_id':tagid}):
            booktag_list.append(tag['tag_name'].split('-'))
    return(booktag_list)

In [ ]:
for book in db.books.aggregate([{'$sample':{'size':5}},{
    '$lookup': {
        'from': 'booktags',
        'localField': 'goodreads_book_id',
        'foreignField': 'goodreads_book_id',
        'as': 'booktags'
    }
}, {
    '$unwind': '$booktags'
}, {
    '$project': {
        '$group': {
            'goodreads_book_id':'$_id',
            'tag_id':'tag_num'
    }   
        'goodreads_book_id': '$goodreads_book_id',
        'tag_num': '$booktags.tag_num'
    }
}
]):
        pprint.pprint(book)

In [ ]:
for doc in db.news.find():
    cc = cleaning(doc['content'])
    db.news.update({'_id':doc['_id']}, {'$set':{'cleaned_content1':cc}})

In [49]:
for book in db.books.find_one():
    print(book['goodreads_book_id'])
    #for book_id in book['goodreads_book_id']:
     #   db.booktags.find_one()

TypeError: string indices must be integers

In [54]:
db.books.find_one()['goodreads_book_id']

2767052

In [55]:
db.booktags.find_one()

{'_id': ObjectId('5b8885c921c3f05eea553c1d'),
 'goodreads_book_id': 1,
 'tag_id': 30574,
 'count': 167697}

In [74]:
new_list = []
for tag in find_booktags(2767052):
    for word in tag:
            new_list.append(word)

In [75]:
new_list

['currently',
 'reading',
 'young',
 'adult',
 'fiction',
 'favorites',
 'to',
 'read',
 'dystopian',
 'fantasy',
 'dystopia',
 'books',
 'i',
 'own',
 'sci',
 'fi',
 'science',
 'fiction',
 'ya',
 'series',
 'owned',
 'favourites',
 'romance',
 'adventure',
 'hunger',
 'games',
 'kindle',
 'book',
 'club',
 'post',
 'apocalyptic',
 'teen',
 'read',
 'in',
 '2012',
 'the',
 'hunger',
 'games',
 'my',
 'books',
 'action',
 'favorite',
 'books',
 'suzanne',
 'collins',
 'all',
 'time',
 'favorites',
 'ya',
 'fiction',
 'survival',
 'sci',
 'fi',
 'fantasy',
 'books',
 'favorite',
 '5',
 'stars',
 'scifi',
 're',
 'read',
 'read',
 'in',
 '2011',
 'i',
 'own',
 'novels',
 'ebook',
 'audiobook',
 'young',
 'adult',
 'fiction',
 'my',
 'favorites',
 'thriller',
 'audiobooks',
 'read',
 '2012',
 'ebooks',
 'library',
 'default',
 'fantasy',
 'sci',
 'fi',
 'shelfari',
 'favorites',
 'futuristic',
 'read',
 'in',
 '2010',
 'favorite',
 'series',
 'faves',
 'my',
 'library',
 'teen',
 'fiction

In [131]:
def find_booktags (ID):
    booktag_list = []
    for book in db.booktags.find({'goodreads_book_id': ID}):
        booktag_list.append(book['tag_id'])
    return booktag_list

In [132]:
find_booktags(1)

[30574,
 11305,
 8717,
 27199,
 11590,
 18886,
 6953,
 11743,
 11557,
 1691,
 24960,
 6857,
 15965,
 6888,
 33114,
 32989,
 22743,
 26785,
 22753,
 6828,
 2104,
 14017,
 9221,
 5207,
 20774,
 7457,
 25765,
 15169,
 3389,
 1128,
 6895,
 17124,
 11505,
 22034,
 11370,
 11497,
 19974,
 3392,
 22973,
 25234,
 29452,
 20849,
 33009,
 10644,
 32623,
 31617,
 5751,
 16427,
 5090,
 25152,
 29011,
 20939,
 24964,
 14024,
 22689,
 33012,
 20824,
 18045,
 6960,
 33124,
 3371,
 21989,
 615,
 32654,
 26897,
 11550,
 18851,
 27535,
 25151,
 11491,
 17213,
 11574,
 25150,
 6838,
 17129,
 30573,
 10197,
 25770,
 30521,
 26138,
 2106,
 33165,
 16799,
 25167,
 8055,
 25153,
 10641,
 1120,
 6920,
 11579,
 22352,
 6914,
 20781,
 26842,
 5444,
 11595,
 18913,
 24092,
 7852,
 6829]

In [130]:
booktag_list = []
for book in db.booktags.find({'goodreads_book_id': 1}):
    booktag_list.append(book['tag_id'])
print (booktag_list)

[30574, 11305, 8717, 27199, 11590, 18886, 6953, 11743, 11557, 1691, 24960, 6857, 15965, 6888, 33114, 32989, 22743, 26785, 22753, 6828, 2104, 14017, 9221, 5207, 20774, 7457, 25765, 15169, 3389, 1128, 6895, 17124, 11505, 22034, 11370, 11497, 19974, 3392, 22973, 25234, 29452, 20849, 33009, 10644, 32623, 31617, 5751, 16427, 5090, 25152, 29011, 20939, 24964, 14024, 22689, 33012, 20824, 18045, 6960, 33124, 3371, 21989, 615, 32654, 26897, 11550, 18851, 27535, 25151, 11491, 17213, 11574, 25150, 6838, 17129, 30573, 10197, 25770, 30521, 26138, 2106, 33165, 16799, 25167, 8055, 25153, 10641, 1120, 6920, 11579, 22352, 6914, 20781, 26842, 5444, 11595, 18913, 24092, 7852, 6829]


In [ ]:
for book in db.booktags.find({'goodreads_book_id':ID}):
        tagid = (book['tag_id'])
        for tag in db.tags.find({'tag_id':tagid}):
            booktag_list.append(tag['tag_name'].split('-'))
    return(booktag_list)

In [67]:
for book in db.booktags.aggregate([{'$sample':{'size':5}}]):
    #print(book)
    #print(book['_id'])
    print({'_id':book['_id']}, {'$set':{'tags': }})

    
#db.news.update({'_id':doc['_id']}, {'$set':{'cleaned_content1':cc}}) 

{'_id': ObjectId('5b8885d321c3f05eea6319a1')}
{'_id': ObjectId('5b8885d221c3f05eea60fc16')}
{'_id': ObjectId('5b8885cb21c3f05eea5832df')}
{'_id': ObjectId('5b8885d021c3f05eea5fa888')}
{'_id': ObjectId('5b8885ce21c3f05eea5be2c5')}


In [110]:
db.booktags.find_one()['goodreads_book_id']

1

In [ ]:
def find_booktags(ID):
    booktag_list = []          
    for book in db.booktags.find({'goodreads_book_id':ID}):
        tagid = (book['tag_id'])
        for tag in db.tags.find({'tag_id':tagid}):
            booktag_list.append(tag['tag_name'].split('-'))
    return(booktag_list)

In [106]:
find_booktags(2767052)

[['currently', 'reading'],
 ['young', 'adult'],
 ['fiction'],
 ['favorites'],
 ['to', 'read'],
 ['dystopian'],
 ['fantasy'],
 ['dystopia'],
 ['books', 'i', 'own'],
 ['sci', 'fi'],
 ['science', 'fiction'],
 ['ya'],
 ['series'],
 ['owned'],
 ['favourites'],
 ['romance'],
 ['adventure'],
 ['hunger', 'games'],
 ['kindle'],
 ['book', 'club'],
 ['post', 'apocalyptic'],
 ['teen'],
 ['read', 'in', '2012'],
 ['the', 'hunger', 'games'],
 ['my', 'books'],
 ['action'],
 ['favorite', 'books'],
 ['suzanne', 'collins'],
 ['all', 'time', 'favorites'],
 ['ya', 'fiction'],
 ['survival'],
 ['sci', 'fi', 'fantasy'],
 ['books'],
 ['favorite'],
 ['5', 'stars'],
 ['scifi'],
 ['re', 'read'],
 ['read', 'in', '2011'],
 ['i', 'own'],
 ['novels'],
 ['ebook'],
 ['audiobook'],
 ['young', 'adult', 'fiction'],
 ['my', 'favorites'],
 ['thriller'],
 ['audiobooks'],
 ['read', '2012'],
 ['ebooks'],
 ['library'],
 ['default'],
 ['fantasy', 'sci', 'fi'],
 ['shelfari', 'favorites'],
 ['futuristic'],
 ['read', 'in', '2010'],

In [ ]:
def get_goodreads_ids():
    '''
    Returns list of 10,000 goodreads books by ID
    '''
    goodreads_id_list = []
    for book in db.books.find():
        goodreads_id_list.append(book['goodreads_book_id'])
    return goodreads_id_list

In [ ]:
def find_booktags (ID):
    '''
    Finds booktag ID numbers associated with each goodread
    '''
    booktag_list = []
    for book in db.booktags.find({'goodreads_book_id': ID}):
        booktag_list.append(book['tag_id'])
    return booktag_list

In [ ]:
def find_booktags(ID):
    booktag_list = []          
    for book in db.booktags.find({'goodreads_book_id':ID}):
        tagid = (book['tag_id'])
        for tag in db.tags.find({'tag_id':tagid}):
            booktag_list.append(tag['tag_name'].split('-'))
    return(booktag_list)

In [133]:
def find_booktags_old(ID):
    '''
    Finds descriptive 'tags' for all goodreads books and looks up associated tag name
    '''
    booktag_list = []          
    for book in db.booktags.find({'goodreads_book_id':ID}):
        tagid = (book['tag_id'])
        for tag in db.tags.find({'tag_id':tagid}):
            try:
                booktag_list.append(tag['tag_name'].split('-'))
            except:
                booktag_list.append(str(tag['tag_name']).split('-'))
    return(booktag_list)

In [134]:
find_booktags_old(1)

[['to', 'read'],
 ['fantasy'],
 ['currently', 'reading'],
 ['series'],
 ['favourites'],
 ['magic'],
 ['childrens'],
 ['fiction'],
 ['favorites'],
 ['adventure'],
 ['re', 'read'],
 ['children'],
 ['j', 'k', 'rowling'],
 ['children', 's'],
 ['young', 'adult'],
 ['ya'],
 ['owned'],
 ['sci', 'fi', 'fantasy'],
 ['owned', 'books'],
 ['childhood'],
 ['all', 'time', 'favorites'],
 ['harry', 'potter'],
 ['default'],
 ['books', 'i', 'own'],
 ['my', 'books'],
 ['classics'],
 ['reread'],
 ['i', 'own'],
 ['audiobook'],
 ['5', 'stars'],
 ['children', 's', 'books'],
 ['kids'],
 ['favorite', 'books'],
 ['novels'],
 ['fantasy', 'sci', 'fi'],
 ['favorite'],
 ['middle', 'grade'],
 ['audiobooks'],
 ['paranormal'],
 ['read', 'more', 'than', 'once'],
 ['teen'],
 ['my', 'library'],
 ['ya', 'fantasy'],
 ['english'],
 ['witches'],
 ['urban', 'fantasy'],
 ['british'],
 ['jk', 'rowling'],
 ['books'],
 ['read', 'in', '2016'],
 ['supernatural'],
 ['mystery'],
 ['re', 'reads'],
 ['harry', 'potter', 'series'],
 ['ow

In [156]:
import pickle

with open('removed_tags.pickle','rb') as read_file:
    removed_tags = pickle.load(read_file)

removed_tag_list = []

for tag in removed_tags:
    for word in tag:
        removed_tag_list.append(word)

stop_words = stopwords.words('english')

removed_words = removed_tag_list + stop_words

In [157]:
def create_booktag_list(ID):
    vector_list = []        
    book_word_list = []
    for tag in find_booktags_old(ID):
        for word in tag:
            if word not in removed_words:
                book_word_list.append(word)
    vector_list.append(book_word_list)
    return vector_list

In [158]:
create_booktag_list(1)

[['fantasy',
  'series',
  'magic',
  'childrens',
  'adventure',
  'children',
  'j',
  'k',
  'rowling',
  'children',
  'young',
  'ya',
  'sci',
  'fi',
  'fantasy',
  'childhood',
  'time',
  'harry',
  'potter',
  'classics',
  'reread',
  '5',
  'stars',
  'children',
  'kids',
  'favorite',
  'fantasy',
  'sci',
  'fi',
  'favorite',
  'middle',
  'grade',
  'paranormal',
  'teen',
  'ya',
  'fantasy',
  'english',
  'witches',
  'urban',
  'fantasy',
  'british',
  'jk',
  'rowling',
  '2016',
  'supernatural',
  'mystery',
  'reads',
  'harry',
  'potter',
  'series',
  'ya',
  'childrens',
  'young',
  'novel',
  '2005',
  'wizards',
  'scifi',
  'fantasy',
  'favorite',
  'series',
  'made',
  'cry',
  'shelfari',
  '2015',
  'faves',
  'favourite',
  '2014',
  'childhood',
  'kids',
  'rereads',
  'romance',
  'time',
  'youth',
  'juvenile',
  'english',
  '2017',
  'england',
  '5',
  'star',
  'children',
  'literature',
  'favourite',
  'shelf',
  'children',
  'lit',


In [162]:
def create_booktag_list(ID):
    book_word_list = []
    for tag in find_booktags_old(ID):
        for word in tag:
            if word not in removed_words:
                book_word_list.append(word)
    return book_word_list

In [163]:
create_booktag_list(1)

['fantasy',
 'series',
 'magic',
 'childrens',
 'adventure',
 'children',
 'j',
 'k',
 'rowling',
 'children',
 'young',
 'ya',
 'sci',
 'fi',
 'fantasy',
 'childhood',
 'time',
 'harry',
 'potter',
 'classics',
 'reread',
 '5',
 'stars',
 'children',
 'kids',
 'favorite',
 'fantasy',
 'sci',
 'fi',
 'favorite',
 'middle',
 'grade',
 'paranormal',
 'teen',
 'ya',
 'fantasy',
 'english',
 'witches',
 'urban',
 'fantasy',
 'british',
 'jk',
 'rowling',
 '2016',
 'supernatural',
 'mystery',
 'reads',
 'harry',
 'potter',
 'series',
 'ya',
 'childrens',
 'young',
 'novel',
 '2005',
 'wizards',
 'scifi',
 'fantasy',
 'favorite',
 'series',
 'made',
 'cry',
 'shelfari',
 '2015',
 'faves',
 'favourite',
 '2014',
 'childhood',
 'kids',
 'rereads',
 'romance',
 'time',
 'youth',
 'juvenile',
 'english',
 '2017',
 'england',
 '5',
 'star',
 'children',
 'literature',
 'favourite',
 'shelf',
 'children',
 'lit',
 'bookshelf',
 'science',
 'fantasy',
 'bookshelf',
 'favs',
 'magical',
 'potter',
 